В блокноте описываются общие подходы к решению различных видов задач на динамическое программирование, встречающихся в курсовой работе по курсу "Теория принятия решений". А именно:

- детерминированные задачи:
  - задачи с дискретным пространством состояний (задача о рюкзаке)
  - задачи с непрерывным пространством состояний (задача об инвестициях)
- недетерминированные задачи (задача о садовнике)

# Введение: динамическое программирование

Итак, динамическое программирование (ДП) - это метод решения многоэтапных задач принятия решений, при котором на каждом шаге управление выбирается в соответствии с принципом Беллмана. А именно: каково бы ни было состояние системы в результате какого-то числа шагов, мы должны выбирать управление на ближайшем шаге так, чтобы оно, в совокупности с оптимальным управлением на всех последующих шагах, приводило к максимальному выигрышу на всех оставшихся шагах, включая данный.

Данный принцип может быть записан с помощью уравнения Беллмана (основного функционального уравнения ДП):

$$
W_i(s_i) = \max_{u_i \in U_i(s_i)} \{ w_i(s_i, u_i) + W_{i+1}(\phi_i(s_i, u_i)) \}
$$

Здесь $W_i(s_i)$ - условно оптимальный выигрыш, то есть, наилучший (максимальный) выигрыш, который может быть получен начиная с $i$-того шага, *при условии*, что система к $i$-тому шагу находится в состоянии $s_i$. $u_i$ - это управление, которое выбирается из множества допустимых управлений $U_i$. Функции $w_i(s_i, u_i)$ и $\phi(s_i, u_i)$ задают выигрыш на $i$-том шаге и функцию изменения состояния соответственно.

В различных источниках по ДП можно встретить разный "взгляд" на данный процесс, связанный с понятиями *cостояние* и *подзадача*. В данном случае, эти понятия обозначают одно и то же. То есть, под *состоянием* и понимается *подзадача*. В качестве примера рассмотрим задачу о рюкзаке, состоящую в том, что из имеющегося набора предметов, каждый из которых обладает определенным весом и определенной стоимостью, необходимо выбрать такое подмножество, суммарный вес которого будет не более заданного (грузоподъемность рюкзака), а стоимость максимальна. При формализации этой задачи (см. лекцию) состояние было определено как остаточная грузоподъемность рюкзака. Соответственно, факт помещения предмета в рюкзак, приводящий к уменьшению грузоподъемности, можно интерпретировать как:
1. переход в другое состояние (с меньшей грузоподъемностью);
2. необходимость решения меньшей задачи (с рюкзаком меньшей грузоподъемности и меньшим набором предметов).

Замечание. Чтобы ДП имело смысл применять к некоторой задаче, она должна обладать определенными свойствами:
1. оптимальная структура подзадач. Это означает, что оптимальное решение задачи достигается при оптимальных решениях подзадач. 
2. пересекающиеся подзадачи. То есть, в ходе декомпозиции исходной задачи на подзадачи одна и та же подзадача может возникать много раз.  

По-хорошему, перед тем, как приступать к решению задачи методом ДП, следует (хотя бы на "интуитивном" уровне) убедиться в том, что задача (и ее разбиение на подзадачи) обладает данными свойствами.

# Детерминированные задачи

Особенностью детерминированных задач является то, что выигрыш от определенного управления $w_i(s_i, u_i)$ и целевое состояние при применении управления $\phi(s_i, u_i)$ являются функциями. То есть, если мы знаем исходное состояние и управление, то выигрыш и целевое состояние определяются однозначно. 

## Задачи с дискретным пространством состояний

В качестве примера такой задачи рассмотрим следующую (из Х. Таха Введение в исследование операций. 6-е издание):

Строительный подрядчик оценивает минимальные потребности в рабочей силе на каждую из последующих пяти недель следующим образом: 5, 6, 8, 4 и 6 рабочих соответственно. Содержание избытка рабочей силы обходится подрядчику в 300 долларов за одного рабочего в неделю, а наем рабочей силы на протяжении одной недели обходится в 400 долларов плюс 200 долларов за одного рабочего в неделю.

In [1]:
# Требуемое количество рабочих для каждого этапа (каждой недели)
workforce_demand = [5, 7, 8, 4, 6]

### Построение модели задачи

При моделировании задачи будем следовать алгоритму моделирования, изложенному в слайдах занятия, посвященного ДП (см. ). 

#### 1. Описание процесса

*Этапы*: в данной задаче есть явная временн*а*я компонента, соответственно, логичней всего связать этапы с временем. То есть, один этап процесса управления соответствует одной неделе.

*Выигрыш*: выигрыш в задаче имеет чисто экономическую основу и связывается с издержками субъекта управления. В данной задаче есть только расходы, соответственно, можно ставить задачу либо как задачу минимизации расходов, либо как максимизации выигрыша (пусть и отрицательного). Остановимся на втором варианте, чтобы не вводить дополнительное понятие "проигрыш".

*Управление*: примем за управление изменение количества рабочих в начале недели (в точке принятия решения). Возможен и более простой вариант, в котором управлением является количество рабочих в определенную неделю.

*Состояние*: количество рабочих.

#### 2. Уравнение выигрыша на $i$-том этапе

Выигрыш определяется всеми экономическими эффектами, связанными с управлением персоналом. А именно:
- наем содрудников (функция $h$);
- содержание избыточной рабочей силы (функция $m$).

$$
w_i(s_i, u_i) = h(u_i) + m(s_i + u_i - d_i).
$$

Здесь $d_i$ - это потребность в рабочих на $i$-том этапе. Обратите внимание, что при таком подходе к моделированию функция $w_i(s_i, u_i)$ оказывается зависимой от этапа (на разных этапах вызов функции с одинаковыми параметрами может дать разный результат, что объясняется различными значениями $d_i$). Чтобы отразить этот факт, в определении функции используется индекс этапа (то есть, де-факто речь идет о целом семействе функций выигрыша). 

Факт наема сотрудников связан с постоянными издержками (400 долларов) и переменными (200 долларов на одного рабочего). Формально это можно записать в следующем виде:

$$
h(u_i) = -(400 + 200u_i) \mathbb{I}[u_i > 0]
$$

Здесь $\mathbb{I}[u_i > 0]$ - индикаторная функция, принимающая значение 1 тогда и только тогда, когда выражение $u_i > 0$ истинно.

Данная функция может быть определена в коде следующим образом:

In [2]:
def hire(state, control):
    return -(400 + 200*control)*int(control > 0)

In [3]:
hire(None, -2)  # Должно быть 0 (увольнения по условию задачи ничего не стоят)

0

In [4]:
hire(None, 0)  # Должно быть 0

0

In [5]:
hire(None, 2)  # Должно быть -400 + (-200) + (-200) = -800

-800

Содержание избыточной рабочей силы обходится в 300 долларов за человека:

$$
m(x) = 300 * x.
$$

Избыточной же считается рабочая сила, превышающая требуемую: $s_i + u_i - d_i$ (количество рабочих к началу этапа плюс количество нанятых рабочих минус требуемое количество).

In [6]:
def excess_workers_maintenance(excess_workers):
    return -300 * excess_workers

In [7]:
# Вычисление выигрыша на заданном этапе.
# В принципе, эта функция может еще включать в себя проверку на допустимость управления
# (во всяком случае, при отладке эта возможность была бы весьма кстати)
def profit(stage, state, control):
    return hire(state, control) + excess_workers_maintenance(state + control - workforce_demand[stage])

In [8]:
profit(0, 0, 6)   # -400 постоянные расходы на наем сотрудников,
                  # -200*6 = -1200 переменные расходы на наем сотрудников
                  # (-300)*(6 - 5) = -300 содержание избыточной рабочей силы
                  # Итого: -1900

-1900

#### 3. Функция перехода для  $i$-того этапа

Факт найма/увольнения сотрудников изменяет количество рабочих очевидным образом:

$$
\phi(s_i, u_i) = s_i + u_i
$$


In [9]:
def state_transition(state, control):
    return state + control

In [10]:
state_transition(0, 5)  # Должно быть 5 (не было сотрудников, наняли пятерых)

5

#### 4. Записываем уравнение Беллмана для данной задачи

Объединяем все компоненты задачи:

$$
W_i(s_i) = \max_{u_i + s_i \geq d_i} 
\{ -(400 + 200u_i) \mathbb{I}[u_i > 0] + 
   (-300)*(s_i + u_i - d_i) + W_{i+1}(s_i + u_i) 
\}
$$



In [11]:

# Количество месяцев
periods = len(workforce_demand)
# Максимальная потребность в рабочей силе
# (используется для определения диапазона допустимых управлений)
max_workers = max(workforce_demand)

# Таблица со значениями функции Беллмана
# Ключом является пара (этап, состояние),
# значением - пара (условный оптимальный выигрыш, условное оптимальное управление)
workforce_W_table = {}

def workforce_W(stage, state):
    """Вычисление функции Беллмана для задачи планирования рабочей силы.
       
       Возвращаемое значение: пара (условный оптимальный выигрыш, условное оптимальное управление)."""

    # Условие выхода из рекурсии
    if stage >= periods:
        return (0, None)

    # Перед перебором управлений проверим, нет ли еще значения для данных
    # параметров в таблице:
    if (stage, state) in workforce_W_table:
        return workforce_W_table[(stage, state)]

    # В соответствии с уравнением Беллмана, найдем условный оптимальный 
    # выигрыш, перебирая допустимые управления и рассчитывая их эффект
    # Диапазон, в котором имеет смысл перебирать управление, ограничен следующим
    # образом.
    # С одной стороны, получающееся количество сотрудников должно быть не меньше, чем
    # требуемое на данном этапе количество.
    # С другой стороны, количество не должно превышать максимальное требуемое количество 
    # рабочих (max_workers).
    # В данном случае, проще перебирать не собственно управления, а состояния, в котором
    # мы можем оказаться:
    best_u = None
    best_W = None
    for workers in range(workforce_demand[stage], max_workers+1):
        # Управление - это разница в количестве рабочих
        control = workers - state
        # Оценка данного управления с учетом последствий (перехода в новое состояние на следующем этапе)
        control_evaluation = profit(stage, state, control) + workforce_W(stage+1, state_transition(state, control))[0]
        # Если это управление лучше, чем наилучшее из известных к данному моменту - запомним его
        if best_W is None or control_evaluation > best_W:
            best_W = control_evaluation
            best_u = control
    # Сохраним найденные значения в таблицу
    workforce_W_table[(stage, state)] = (best_W, best_u)
    return (best_W, best_u)
    

In [12]:
workforce_W(0, 0)

(-3300, 5)

То есть, оптимальный план соответствует выигрышу -3300 долларов и для реализации этого плана нужно в первый месяц нанять пятерых рабочих.

А как узнать условные оптимальные управления для других этапов? Они уже содержатся в таблице `workforce_W_table`, необходимо только извлечь их оттуда.

Вспомним, что каждая запись таблицы отображает ключ (*номер этапа*, *количество рабочих*) в пару (*условно оптимальный выигрыш на этом и всех последующих этапах*, *условно оптимальное управление*). Это значит, что первым шагом в цепочке оптимального управления является управление 5, а второй шаг определяется тем, в каком состоянии мы окажемся в результате применения этого управления. Но для определения этого состояния мы можем воспользоваться уже имеющейся функцией перехода - `state_transition`!

In [13]:
# Из состояния 0 на этапе 0 (первая неделя) под действием управления 5 мы перейдем, очевидно, в состояние 5
state_transition(0, 5)

5

In [14]:
# На этапе 1 (вторая неделя) для состояния 5 оптимальным является управление 3
workforce_W_table[(1, 5)]

(-1900, 3)

In [15]:
# Из состояния 5 на этапе 1 (вторая неделя) под действием управления 3 мы перейдем в состояние 8
state_transition(5, 3)

8

И так далее. То есть, вся последовательность оптимального управления восстанавливается с помощью моделирования переходов. Этот процесс можно обобщить с помощью следующей функции:

In [16]:
def restore_optimal_control(from_stage, to_stage, state):
    """Восстановление оптимального управления из заданного состояния на заданном этапе."""
    optimal_control_sequence = []
    for stage in range(from_stage, to_stage):
        _, control = workforce_W_table[(stage, state)]
        # Запоминаем управление
        optimal_control_sequence.append(control)
        # Моделируем переход под воздействием этого управления
        state = state_transition(state, control)
    return optimal_control_sequence    

restore_optimal_control(0, 5, 0)

[5, 3, 0, -2, 0]

То есть, оптимальное управление заключается в том, чтобы на первой и второй неделях нанять по 5 и 3 рабочих соответственно, на четвертой неделе двоих уволить, а на третьей и пятой неделях ничего не менять.

Очень полезно для самопроверки реализовать функцию оценки заданного управления (например, чтобы с помощью нескольких тестов убедиться в том, что найденное решение лучше многих других):

In [17]:
def evaluate_control_sequence(from_stage, initial_state, sequence):
    stage = from_stage
    state = initial_state
    cost = 0
    for control in sequence:
        # Эффект заданного управления в заданном состоянии
        cost += profit(stage, state, control)
        # Новое состояние
        state = state_transition(state, control)
        stage += 1
    return cost

Проверим оценку оптимального управления:

In [18]:
evaluate_control_sequence(0, 0, restore_optimal_control(0, 5, 0))

-3300

Действительно, стоимость этого плана - 3300 долларов. Попробуем несколько других планов:

In [19]:
# Найм максимального количества рабочих в первую же неделю
evaluate_control_sequence(0, 0, [max_workers, 0, 0, 0, 0])

-5000

In [20]:
# Содержание только необходимого количества рабочих в каждый месяц
evaluate_control_sequence(0, 0, [5, 2, 1, -4, 2])

-3600

Видим, что обе тривиальные стратегии оказались хуже, чем найденная оптимальная. Конечно, в строгом смысле это не доказывает ни оптимальность стратегии, найденной методом ДП, ни корректность реализации, однако позволяет провести хотя бы базовую проверку разумности.

## Детерминированные задачи с непрерывным состоянием

См. [другой блокнот](Dynamic%20Programming%20(continuous).ipynb).


# Недетерминированные задачи

Особенностью таких задач является то, что при известном состоянии на $i$-том шаге $S_i$ и известном управлении $u_i$, состояние на следующем ($i+1$) шаге все равно связано с некоторой неопределенностью (как правило, раскрываемой с помощью аппарата теории вероятностей).

Классическим примером такого рода задач является задача о садовнике (из Х. Таха Введение в исследование операций, 6-е издание).

Каждый год в начале сезона садовник проводит химический анализ состояния почвы в своем саду. В зависимости от результатов анализа продуктивность сада на новый сезон оценивается как: 1) хорошая, 2) удовлетворительная или 3) плохая.

В результате наблюдений на протяжении многих лет, садовник заметил, что продуктивность в текущем году зависит только от состояния почвы в предыдущем году. Поэтому вероятности перехода почвы из одного состояния продуктивности в другое для каждого года можно представить как следующую матрицу вероятностей перехода:

$$
P^1 = 
\begin{pmatrix}
0.2 & 0.5 & 0.3\\
0 & 0.5 & 0.5 \\
0 & 0 & 1 
\end{pmatrix}.
$$

(Элемент матрицы в строке $i$ и столбце $j$ показывает вероятность перехода почвы из состояния $i$ в состояние $j$. Например, вероятность перехода почвы из состояния "хорошая продуктивность" (строка 1) в состояние "плохая продуктивность" (столбец 3) за 1 сезон оценивается как 0.3. Поскольку в рамках данной модели существует всего три возможных состояния почвы, сумма значений в каждой строке должна быть равна 1.)

В результате различных агротехнических мероприятий садовник может изменить переходные вероятности $P^1$. Обычно для повышения продуктивности почвы применяются удобрения. Эти мероприятия приводят к новой матрице переходных вероятностей $P^2$:

$$
P^2 = 
\begin{pmatrix}
0.3 & 0.6 & 0.1\\
0.1 & 0.6 & 0.3 \\
0.05 & 0.4 & 0.55 
\end{pmatrix}.
$$

Чтобы рассмотреть задачу принятия решений в перспективе, садовник связывает с переходом из одного состояния почвы в другое функцию дохода (или структуру вознаграждения), которая определяет прибыль или убыток за одногодичный период в зависимости от состояний, между которыми осуществляется переход. Так как садовник может принять решение использовать или не использовать удобрения, его доход или убыток будет измениться в зависимости от принятого решения. Матрицы $R^1$ и $R^2$ определяют функции дохода (в сотнях долларов) и соответствуют матрицам переходных вероятностей $P^1$ и $P^2$:


$$
R^1 = 
\begin{pmatrix}
7 & 6 & 3 \\
0 & 5 & 1 \\
0 & 0 & -1 
\end{pmatrix},
$$


$$
R^2 = 
\begin{pmatrix}
6 & 5 & -1 \\
7 & 4 & 0 \\
6 & 3 & -2 
\end{pmatrix}.
$$

Элементы $r_{ij}^2$ матрицы $R^2$ учитывают затраты, связанные с применением удобрения. Например, если система находится в состоянии 1 и остается в этом состоянии и в следующем году, то доход составит $r_{11}^2 = 6$, если же удобрения не используются, то $r_{11}^1 = 7$. 

В конечном итоге, садовник хочет выработать стратегию поведения (вносить удобрения или не вносить), позволяющую добиться *максимального дохода*.

Поскольку конкретное значение дохода от реализуемой стратегии внесения удобрений характеризуется неопределенностью, речь идет о максимизации *ожидаемого дохода* (математического ожидания дохода). (Эта подмена, в частности, является следствием применения теории полезности фон Неймана - Моргенштерна, одним из наиболее распространенных способов сравнивать полезность событий, обладающих разными вероятностями.)

## Решение задачи "от базовых принципов"

Рассмотрим вариант задачи о садовнике, в котором ставится задача максимизации ожидаемого дохода за ограниченный период времени $N$ лет. То есть, садовник хочет выработать такую стратегию внесения удобрений, при которой его ожидаемый доход за определенный период был бы максимален. (При этом вполне возможно, что к концу периода почва окажется в очень "невыгодном" состоянии.)

Оказывается, что для решения такой задачи вполне подходит принцип ДП. Однако для учета неопределенности переходов в модель ДП, рассмотренную выше, нужно ввести несколько уточнений.

Пусть $s_i \in S_i$ - состояние почвы на $i$-том этапе (в $i$-тый год), а $u_i$ - управление, применяемое в $i$-тый год ($u_i = 1$ означает, что удобрения не вносятся, а $u_i = 2$ означает, что вносятся).

Уточнение 1. В детерминированных задачах переход из некоторого состояния $s_i$ на $i$-том этапе под действием управления $u_i$ задавался *функцией* $\phi(s_i, u_i)$. То есть, при заданном исходном состоянии и заданном управлении целевое состояние можно было определить однозначно. В данном случае это не так. При любом управлении потенциально может быть возможен переход из любого состояния в любоей (возможно, с разными вероятностями). Поэтому вместо функции $\phi(s_i, u_i)$ имеет смысл рассматривать функцию вероятности перехода $\phi(s_i, s_{i+1}, u_i)$ (по сути, определяемую матрицей перехода вероятностей $P^{u_i}$).

Уточнение 2. В детерминированных задачах выигрыш в состоянии $s_i$ под действием управления $u_i$ также задавался *функцией* $w(s_i, u_i)$. В данной задаче и это не так. Выигрыш зависит не только от того, какое управление мы применили, но и в какое состояние систему "занесло" в силу воздействия совокупности факторов, оценить которые мы можем только вероятностно. То есть, на смену функции выигрыша $w(s_i, u_i)$ приходит функция выигрыша $w(s_i, s_{i+1}, u_i)$ (по сути, это матрица стоимостей переходов $R^{u_i}$).

Уточнение 3. Ожидаемый выигрыш при условии, что система к $i$-тому этапу находится в состоянии $S_i$, также имеет рекуррентную природу, но уравнение Беллмана записывается уже для максимизации математического ожидания:

$$
W_i(s_i) = \max_{u_i \in U_i(s_i)} \{ \sum_{s_j \in S_{i+1}} \phi_i(s_i, s_j, u_i) (w_i(s_i, s_j, u_i) + W_{i+1}(s_j)) \}
$$

То есть, условный оптимальный ожидаемый выигрыш на всех этапах, начиная с $i$-того, при условии, что $i$-тому этапу система находится в состоянии $s_i$, определяется как выбор такого управления $u_i$ из множества допустимых управлений $U_i(s_i)$, при котором максимизируется математическое ожидание суммы стоимости перехода и ожидаемого условного оптимального выигрыша на последующих этапах. Здесь $S_{i+1}$ - множество тех состояний, в которых система может оказаться на этапе $i+1$ (в задаче о садовнике множество состояний одинаково на всех этапах, но это может быть и не так).

Модифицируем программную реализацию функции Беллмана для максимизации ожидаемого выигрыша. В первую очередь, зададим матрицы, описывающие вероятность и стоимости переходов:

In [21]:
import numpy as np

# Матрицы переходов из условия задачи
P1 = np.array([[0.2, 0.5, 0.3],
               [  0, 0.5, 0.5],
               [  0,   0,   1]])

P2 = np.array([[ 0.3, 0.6,  0.1],
               [ 0.1, 0.6,  0.3],
               [0.05, 0.4, 0.55]])

def assert_transition_probabilities(m):
    """Проверяет, является ли заданная матрица правильной матрицей вероятностей переходов."""
    if np.any(abs(m.sum(axis=1) - 1) > 1e-9):
        raise Exception('Each row has to sum to 1.')

assert_transition_probabilities(P1)
assert_transition_probabilities(P2)

# Матрицы стоимостей переходов из условия задачи

R1 = np.array([[7, 6,  3],
               [0, 5,  1],
               [0, 0, -1]])
R2 = np.array([[6, 5, -1],
               [7, 4,  0],
               [6, 3, -2]])


Чтобы реализация вычисления функции Беллмана была максимально похожа на реализацию для детерминированного случая, определим функции выигрыша и вероятности перехода. В принципе, этого можно и не делать и работать в функции Беллмана напрямую с матрицами:

In [22]:
def profit(state_from, state_to, control):
    if control == 1:
        return R1[state_from, state_to]
    elif control == 2:
        return R2[state_from, state_to]
    else:
        raise Exception(f'Invalid control: {control}')
        
def state_transition_probability(state_from, state_to, control):
    if control == 1:
        return P1[state_from, state_to]
    elif control == 2:
        return P2[state_from, state_to]
    else:
        raise Exception(f'Invalid control: {control}')

Собственно, вычисление функции Беллмана:

In [23]:
# Количество лет (горизонт планирования)
gardener_periods = 3

# Таблица со значениями функции Беллмана
# Ключом является пара (этап, состояние),
# значением - пара (ожидаемый условный оптимальный выигрыш, условное оптимальное управление)
gardener_W_table = {}

def gardener_W(stage, state):
    """Вычисление функции Беллмана для задачи о садовнике.
       
       Возвращаемое значение: пара (ожидаемый условный оптимальный выигрыш, условное оптимальное управление)."""

    # Условие выхода из рекурсии
    if stage >= gardener_periods:
        return (0, None)

    # Перед перебором управлений проверим, нет ли еще значения для данных
    # параметров в таблице:
    if (stage, state) in gardener_W_table:
        return gardener_W_table[(stage, state)]

    # В соответствии с уравнением Беллмана для вероятностных задач, найдем 
    # ожидаемый условный оптимальный выигрыш, перебирая допустимые управления,
    # и рассчитывая математическое ожидание их эффекта с учетом вероятностей переходов.
    best_u = None
    best_W = None
    for control in [1, 2]:
        # 1 - не вносить удобрения, 2 - вносить
        
        # Потенциально, при любом управлении мы можем перейти в любое
        # состояние, поэтому здесь добавляется еще один цикл для оценки
        # математического ожидания:
        accumulated_sum = 0
        for new_state in range(3):
            accumulated_sum += state_transition_probability(state, new_state, control) * \
                                    (profit(state, new_state, control) + gardener_W(stage+1, new_state)[0])

        # Если это управление лучше, чем наилучшее из известных к данному моменту - запомним его
        if best_W is None or accumulated_sum > best_W:
            best_W = accumulated_sum
            best_u = control
            
    # Сохраним найденные значения в таблицу
    workforce_W_table[(stage, state)] = (best_W, best_u)
    return (best_W, best_u)


Получим оценку ожидаемого условного оптимального выигрыша (если в начале процесса почва была в хорошем состоянии):

In [24]:
gardener_W(0, 0)

(10.7355, 2)

То есть, в этом случае наибольший ожидаемый выигрыш будет равен приблизительно 10.74 сотен долларов.

### Восстановление оптимальной стратегии

В детерминированной модели мы могли полностью восстановить всю цепочку управлений (поскольку достоверно знали в каком состоянии система окажется в результате каждого из них). Здесь так сделать не получится - действительно, после того, как садовник применил оптимальное управление в первый год (осуществил подкормку), почва могла перейти в любое из трех состояний, и заранее сказать каким должно быть управление во второй год мы уже не можем.

Поэтому для подобных задач результатом является полная *стратегия* оптимального управления, для каждого этапа и каждого состояния определяющая оптимальное действие (действие, приводящее к наибольшему ожидаемому выигрышу).

Как и в предыдущем случае, данная стратегия уже содержится в словаре `gardener_W_table`, необходимо только извлечь ее оттуда. Для этого можно воспользоваться как самим словарем, так и функцией `gardener_W` которая все равно не будет рассчитывать значений, поскольку они уже содержатся в словаре. Итак, оптимальная стратегия для первого года:

In [25]:
state_decoder = ['хорошее', 'удовлетворительное', 'плохое']
control_decoder = [None, 'Не применять удобрения', 'Применять удобрения']

print('Стратегия для 1-го года:')
for state in range(3):
    expected_profit, control = gardener_W(0, state)
    print(f'  При состоянии почвы "{state_decoder[state]}": "{control_decoder[control]}" -> {expected_profit}')

Стратегия для 1-го года:
  При состоянии почвы "хорошее": "Применять удобрения" -> 10.7355
  При состоянии почвы "удовлетворительное": "Применять удобрения" -> 7.922499999999999
  При состоянии почвы "плохое": "Применять удобрения" -> 4.22225


Аналогично находится стратегия для 2-го года:

In [26]:
print('Стратегия для 2-го года:')
for state in range(3):
    expected_profit, control = gardener_W(1, state)
    print(f'  При состоянии почвы "{state_decoder[state]}": "{control_decoder[control]}" -> {expected_profit}')

Стратегия для 2-го года:
  При состоянии почвы "хорошее": "Применять удобрения" -> 8.19
  При состоянии почвы "удовлетворительное": "Применять удобрения" -> 5.61
  При состоянии почвы "плохое": "Применять удобрения" -> 2.125


И для третьего года:

In [27]:
print('Стратегия для 3-го года:')
for state in range(3):
    expected_profit, control = gardener_W(2, state)
    print(f'  При состоянии почвы "{state_decoder[state]}": "{control_decoder[control]}" -> {expected_profit}')

Стратегия для 3-го года:
  При состоянии почвы "хорошее": "Не применять удобрения" -> 5.300000000000001
  При состоянии почвы "удовлетворительное": "Применять удобрения" -> 3.1
  При состоянии почвы "плохое": "Применять удобрения" -> 0.40000000000000013


### Моделирование стратегии

В качестве самопроверки может оказаться полезным промоделировать ту или иную стратегию управления и определить ожидаемый выигрыш от следования ей.

Процесс может быть организован точно так же, как и вычисление функции Беллмана, только на каждом этапе нет необходимости осуществлять оптимизацию (выбор управления), требуется просто применять заданное управление.


In [28]:
# Внимание! Данная функция очень неэффективна без мемоизации. Лучше делать то же самое, но
# в обратном порядке и без рекурсии
def gardener_eval_strategy(strategy, state):
    if len(strategy) == 0:
        return 0
    
    # Управление, диктуемое текущей стратегией для данного состояния
    control = strategy[0][state]
    
    accumulated_cost = 0
    for new_state in range(3):
        accumulated_cost += state_transition_probability(state, new_state, control) * \
                                    (profit(state, new_state, control) + gardener_eval_strategy(strategy[1:], new_state))
    return accumulated_cost

Оценим эффект "скупой" стратегии, при которой удобрения не вносятся ни при каких условиях:

In [29]:
gardener_eval_strategy([[1, 1, 1],   # Управление для каждого из состояний на первом этапе
                        [1, 1, 1],   # Управление для каждого из состояний на втором этапе
                        [1, 1, 1]],  # ...
                       0)

8.212000000000002

То есть, даже если мы начнем с почвы в "хорошем" состоянии, ожидаемый выигрыш будет около 8.2.

Оценим эффект "расточительной" стратегии, при которой удобрения вносятся безусловно:

In [30]:
gardener_eval_strategy([[2, 2, 2],
                        [2, 2, 2],
                        [2, 2, 2]],
                       0)

10.6425

Видно, что при данной стратегии ожидаемый выигрыш оказывается очень близким к оптимальному. Это вполне ожидаемо, учитывая, что оптимальная стратегия почти совпадает с данной (единственное отличие заключается в том, что в оптимальной стратегии на третьем году в хорошую почву удобрения не вносятся).

### Матричная реализация

TODO

## Сведение задачи к марковскому процессу принятия решений

TODO